In [1]:
import numpy as np
import pandas
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
from PIL import Image

In [2]:
train_data_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/train"
test_data_dir = "/kaggle/input/chest-xray-pneumonia/chest_xray/test"
BATCH_SIZE = 32
num_classes = 2

learning_rate = 0.0001

In [3]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)  # Add validation split

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(150, 150),
                                                    class_mode='categorical',
                                                    color_mode='rgb',  # Ensure images are loaded in RGB format
                                                    subset='training')  # Specify 'training' subset

validation_generator = train_datagen.flow_from_directory(train_data_dir,
                                                         target_size=(150, 150),
                                                         class_mode='categorical',
                                                         color_mode='rgb',  # Ensure images are loaded in RGB format
                                                         subset='validation')  # Specify 'validation' subset

test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(150, 150),
                                                  class_mode='categorical',
                                                  color_mode='rgb')  # Ensure images are loaded in RGB format

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples

Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
pre_trained_weights = '/kaggle/input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
base_model = InceptionV3(weights=pre_trained_weights, include_top=False, input_shape=(150, 150, 3))
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Run the model
model.fit(train_generator,epochs=10,steps_per_epoch=nb_train_samples // BATCH_SIZE,validation_data=validation_generator,verbose = 1,validation_steps=nb_validation_samples // BATCH_SIZE)

Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


130/130 ━━━━━━━━━━━━━━━━━━━━ 197s 1s/step - accuracy: 0.8807 - loss: 0.3895 - val_accuracy: 0.8975 - val_loss: 0.2828
Epoch 2/10
  1/130 ━━━━━━━━━━━━━━━━━━━━ 1:30 700ms/step - accuracy: 1.0000 - loss: 0.0461

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 1.0000 - loss: 0.0461 - val_accuracy: 0.8421 - val_loss: 0.4762
Epoch 3/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.9212 - loss: 0.2275 - val_accuracy: 0.9316 - val_loss: 0.1713
Epoch 4/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9688 - loss: 0.1050 - val_accuracy: 0.8947 - val_loss: 0.3090
Epoch 5/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 158s 1s/step - accuracy: 0.9247 - loss: 0.1894 - val_accuracy: 0.9404 - val_loss: 0.1659
Epoch 6/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9375 - loss: 0.1381 - val_accuracy: 0.8947 - val_loss: 0.1523
Epoch 7/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.9407 - loss: 0.1470 - val_accuracy: 0.9219 - val_loss: 0.1978
Epoch 8/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9375 - loss: 0.1568 - val_accuracy: 0.8947 - val_loss: 0.3487
Epoch 9/10
130/130 ━━━━━━━━━━━━━━━━━━━━ 159s 1s/step - accuracy: 0.9407 - loss: 0.1571 - val_accuracy: 0.9150 - 

In [6]:
print("Model evaluation on test data:")
model.evaluate(test_generator, steps=len(test_generator))

# Save the trained model
model.save('image_classifier_model_with_inceptionv3.h5')

Model evaluation on test data:
20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 799ms/step - accuracy: 0.8766 - loss: 0.3491
